# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import patsy

In [53]:
df = pd.read_csv('previsao_de_renda.csv')

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

### 1 - Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [100]:
df.sample(1)

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
10661,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,59,7.722635,1.0,2777.15


In [56]:
df.isnull().value_counts()

Unnamed: 0  data_ref  id_cliente  sexo   posse_de_veiculo  posse_de_imovel  qtd_filhos  tipo_renda  educacao  estado_civil  tipo_residencia  idade  tempo_emprego  qt_pessoas_residencia  renda
False       False     False       False  False             False            False       False       False     False         False            False  False          False                  False    12427
                                                                                                                                                    True           False                  False     2573
dtype: int64

In [57]:
#definindo os dados faltantes com a media do dataframe
df['tempo_emprego'].fillna(df['tempo_emprego'].mean(), inplace=True);

In [58]:
df.isnull().value_counts()

Unnamed: 0  data_ref  id_cliente  sexo   posse_de_veiculo  posse_de_imovel  qtd_filhos  tipo_renda  educacao  estado_civil  tipo_residencia  idade  tempo_emprego  qt_pessoas_residencia  renda
False       False     False       False  False             False            False       False       False     False         False            False  False          False                  False    15000
dtype: int64

In [59]:
#remove as tres primeiras colunas 'Unnamed' 'data_ref' e 'id_cliente'
df = df.iloc[:, 3:]

In [60]:
df.head(1)

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.60274,1.0,8060.34


In [61]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

In [62]:
df_train.size, df_test.size

(135000, 45000)

### 2 - Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o  R2  na base de testes. Qual o melhor modelo?

In [43]:
import statsmodels.api as sm

In [44]:
modelo = '''np.log(renda) ~ posse_de_veiculo
              +posse_de_imovel
              +qtd_filhos
              +tipo_renda
              +educacao
              +estado_civil
              +tipo_residencia
              +idade
              +tempo_emprego
              +qt_pessoas_residencia'''

y_train, X_train = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test, X_test = patsy.dmatrices(formula_like=modelo, data=df_test)

In [45]:
alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []
for i in alpha:

  md = sm.OLS(y_train,  X_train)
  reg = md.fit_regularized(method='elastic_net',
                          refit=True,
                          L1_wt = 0,
                          alpha = i)

  y_pred = reg.predict(X_test)
  r2 = r2_score(y_test, y_pred)

  r2_pred.append(r2)
  print('Alpha: ',i, '\nR-squared: ', r2, '\n')


Alpha:  0 
R-squared:  0.20044711780995272 

Alpha:  0.001 
R-squared:  0.19347061803606247 

Alpha:  0.005 
R-squared:  0.17094263187952474 

Alpha:  0.01 
R-squared:  0.13990885084892368 

Alpha:  0.05 
R-squared:  -0.1414303952490843 

Alpha:  0.1 
R-squared:  -0.4249032732684561 



###### O melhor modelo foi o com Alpha: 0  com o  R-squared = 0.20044711780995272




### 3 - Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [46]:
alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []
for i in alpha:

  md = sm.OLS(y_train,  X_train)
  reg = md.fit_regularized(method='elastic_net',
                          refit=True,
                          L1_wt = 1,
                          alpha = i)

  y_pred = reg.predict(X_test)
  r2 = r2_score(y_test, y_pred)

  r2_pred.append(r2)
  print('Alpha: ',i, '\nR-squared: ', r2, '\n')

Alpha:  0 
R-squared:  0.20044711780995306 

Alpha:  0.001 
R-squared:  0.19959077449382323 

Alpha:  0.005 
R-squared:  0.19613115356695865 

Alpha:  0.01 
R-squared:  0.19497948731786363 

Alpha:  0.05 
R-squared:  0.18135066814860745 

Alpha:  0.1 
R-squared:  0.18034692442687172 



###### Com o alpha 0 os metodos são semelhantes com uma minima diferença, com o alpha mais alto há uma diferença maior e o melhor metodo é o *LASSO*.


### 4 - Rode um modelo stepwise. Avalie o  R2  na base de testes. Qual o melhor resultado?

In [47]:
df_dummies = pd.get_dummies(data=df.dropna(), drop_first=True).astype(int)

X = df_dummies.drop(columns='renda')
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1993)

In [48]:
def stepwise(X, y,
             initial_list=[],
             threshold_in=0.05,
             threshold_out=0.05,
             verbose=True):
   included = list(initial_list)
   while True:
        changed=False

        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True

        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)

        if not changed:
            break
        return included

variaveis = stepwise(X_test, y_test)

In [49]:
alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []
for i in alpha:

  md = sm.OLS(y_train,  X_train)
  reg = md.fit_regularized(method='elastic_net',
                          refit=True,
                          L1_wt = 1,
                          alpha = i)

  y_pred = reg.predict(X_test)
  r2 = r2_score(y_test, y_pred)

  r2_pred.append(r2)
  print('Alpha: ',i, '\nR-squared: ', r2, '\n')

Alpha:  0 
R-squared:  0.2520214436314304 

Alpha:  0.001 
R-squared:  0.2520214436314304 

Alpha:  0.005 
R-squared:  0.2520214436314304 

Alpha:  0.01 
R-squared:  0.2520214436314304 

Alpha:  0.05 
R-squared:  0.2520214436314304 

Alpha:  0.1 
R-squared:  0.2521384744120252 



In [50]:
alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []
for i in alpha:

  md = sm.OLS(y_train,  X_train)
  reg = md.fit_regularized(method='elastic_net',
                          refit=True,
                          L1_wt = 0,
                          alpha = i)

  y_pred = reg.predict(X_test)
  r2 = r2_score(y_test, y_pred)

  r2_pred.append(r2)
  print('Alpha: ',i, '\nR-squared: ', r2, '\n')

Alpha:  0 
R-squared:  0.25202144363143053 

Alpha:  0.001 
R-squared:  0.2520447282907633 

Alpha:  0.005 
R-squared:  0.25174957216906624 

Alpha:  0.01 
R-squared:  0.25118652527735896 

Alpha:  0.05 
R-squared:  0.24519901871278993 

Alpha:  0.1 
R-squared:  0.23730885572202598 



###### O modelo lasso se saiu melhor tendo o msmo R² em todos os valores de Alphas

### 5 - Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

###### O melhor modelo foi usando o metodo *lasso* pois obteve melhor coeficiente no R² em todos os valores de Alpha.

### 6 - Partindo dos modelos que você ajustou, tente melhorar o  R2  na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [126]:
modelo = '''np.log(renda) ~ posse_de_veiculo
              +sexo
              +posse_de_imovel
              +qtd_filhos
              +tipo_renda
              +educacao == 'Superior completo'
              +estado_civil
              +idade
              +tempo_emprego
              +qt_pessoas_residencia'''

y_train, X_train = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test, X_test = patsy.dmatrices(formula_like=modelo, data=df_test)

In [127]:
alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []
for i in alpha:

  md = sm.OLS(y_train,  X_train)
  reg = md.fit_regularized(method='elastic_net',
                          refit=True,
                          L1_wt = 0,
                          alpha = i)

  y_pred = reg.predict(X_test)
  r2 = r2_score(y_test, y_pred)

  r2_pred.append(r2)
  print('Alpha: ',i, '\nR-squared: ', r2, '\n')

Alpha:  0 
R-squared:  0.3526806719657687 

Alpha:  0.001 
R-squared:  0.35047833098175984 

Alpha:  0.005 
R-squared:  0.33547910688778393 

Alpha:  0.01 
R-squared:  0.3022034767368549 

Alpha:  0.05 
R-squared:  -0.03269239703834925 

Alpha:  0.1 
R-squared:  -0.334425835887751 



### 7 - Ajuste uma árvore de regressão e veja se consegue um  R2  melhor com ela.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1993)

regr = DecisionTreeRegressor(max_depth=4)
regr.fit(X_train, y_train);

In [178]:
regr.score(X=X_test, y=y_test)

0.44713459926804155